In [ ]:
f = open('1435449602.xml', 'r')
article = f.read()
f.close()

## 將XML 資料讀取進成xml 架構

In [2]:
from xml.etree import ElementTree
with open('1435449602.xml','r',encoding='utf-8') as f:
    events=ElementTree.fromstring(f.read())

type(events)

xml.etree.ElementTree.Element

## 從item 之中分別取得標題與內文，並進行切詞

In [12]:
import jieba, re

corpus = []
titles = []
for elem in events.findall('./channel/item'):
    title       = elem.find('title').text
    description = elem.find('description').text
    titles.append(title)
    article = []
    for word in jieba.cut(description):
        if re.match('[\u4e00-\u9fa5]+', word):
            article.append(word)
    corpus.append(' '.join(article))

## 建立詞頻矩陣

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [14]:
X.shape

(147, 12247)

In [16]:
word = vectorizer.get_feature_names()
#word

## 建立 tf-idf 矩陣

In [17]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
tfidf       = transformer.fit_transform(X)
tfidf.toarray()

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [19]:
tfidf.shape

(147, 12247)

## 使用Kmeans 進行分群

In [21]:
from sklearn import cluster
#?cluster.KMeans
c=cluster.KMeans(n_clusters=4)

In [22]:
weight = tfidf.toarray()
k_data = c.fit_predict(weight)

## 使用Enumerate 取出分群結果

In [24]:
a= [2,5,3,4,7]
for idx, e in enumerate(a):
    print(idx, e)

0 2
1 5
2 3
3 4
4 7


In [40]:
for pos, group in enumerate(k_data):
    if group == 1:
        print(titles[pos])

蔡依林淚奪金曲 錦榮傳訊恭喜
陳奕迅、張惠妹稱王封后  蔡依林抱回最大獎
陳奕迅、莫文蔚伴侶均不知阿娜答金曲獲獎
金曲26／蔡依林擒３獎大勝　淚崩再挺婚姻平權
金曲26／張惠妹奪歌后卻失落　要世界感受彩虹力量
金曲26／蔡依林淚奪最佳專輯＋完整得獎名單
僅次Jolin！徐佳瑩入圍6獎全槓被封遺珠
金曲最風光！蔡依林紅毯全勝又獲3獎成大贏家
張惠妹3度封后  想破江蕙紀錄
金曲26／陳奕迅稱王謝台灣　張惠妹封后秒噴淚
蔡依林呸大贏家  金曲最佳專輯獎
金曲獎完整得獎名單！阿妹封后 陳奕迅稱王
第26屆金曲獎 陳奕迅奪歌王、阿妹封歌后
金曲最佳國語專輯：呸
《金曲26》2015金曲獎得獎名單 線上直播懶人包


## 使用Numpy 取出分群結果

In [39]:
import numpy
new_titles = numpy.array(titles)
new_titles[k_data == 1]

array(['蔡依林淚奪金曲 錦榮傳訊恭喜', '陳奕迅、張惠妹稱王封后  蔡依林抱回最大獎', '陳奕迅、莫文蔚伴侶均不知阿娜答金曲獲獎',
       '金曲26／蔡依林擒３獎大勝\u3000淚崩再挺婚姻平權', '金曲26／張惠妹奪歌后卻失落\u3000要世界感受彩虹力量',
       '金曲26／蔡依林淚奪最佳專輯＋完整得獎名單', '僅次Jolin！徐佳瑩入圍6獎全槓被封遺珠',
       '金曲最風光！蔡依林紅毯全勝又獲3獎成大贏家', '張惠妹3度封后  想破江蕙紀錄',
       '金曲26／陳奕迅稱王謝台灣\u3000張惠妹封后秒噴淚', '蔡依林呸大贏家  金曲最佳專輯獎',
       '金曲獎完整得獎名單！阿妹封后 陳奕迅稱王', '第26屆金曲獎 陳奕迅奪歌王、阿妹封歌后', '金曲最佳國語專輯：呸',
       '《金曲26》2015金曲獎得獎名單 線上直播懶人包'], 
      dtype='<U26')

## 取出分群結果關鍵字

In [54]:
import numpy as np
np.sum([
        [0, 1], 
        [0, 5]
       ], axis=0)

array([0, 6])

In [77]:
#group1         = tfidf[k_data == 2].toarray()
group1         = tfidf[k_data == 1].toarray()

# 計算colsums
group1_colsums = np.sum(group1, axis = 0)

# 根據colsums 做由大到小的排序
swd = group1_colsums.argsort()[::-1]

# 將每個column keyword 變成numpy array
keywords = numpy.array(vectorizer.get_feature_names())
for term, cnt in zip(keywords[swd][0:15], group1_colsums[swd][0:15]):
    print(term, cnt)

最佳 4.04617459463
專輯 2.82520550006
金曲 1.94067007063
蔡依林 1.64389056436
國語 1.58337074329
有限公司 1.2177551509
音樂 1.20907925857
張惠妹 1.1492862655
開心 1.10664627155
演唱 0.963153943651
股份 0.927744639562
專輯獎 0.921736618957
獲得 0.874320017843
歌曲 0.844433460573
典禮 0.781024150118


In [53]:
word = numpy.array(vectorizer.get_feature_names())
word[np.sum(tfidf[k_data == 1].toarray(), axis = 0).argsort()[::-1]][1:10]

array(['專輯', '金曲', '蔡依林', '國語', '有限公司', '音樂', '張惠妹', '開心', '演唱'], 
      dtype='<U7')

## 抓取各群關鍵字函式

In [90]:

def getGroupKeyword(group_id, top_k):
    group1         = tfidf[k_data == group_id].toarray()

    # 計算colsums
    group1_colsums = np.sum(group1, axis = 0)

    # 根據colsums 做由大到小的排序
    swd = group1_colsums.argsort()[::-1]

    # 將每個column keyword 變成numpy array
    keywords = numpy.array(vectorizer.get_feature_names())
    for term, cnt in zip(keywords[swd][0:top_k], group1_colsums[swd][0:top_k]):
        print(group_id, term, cnt)

In [91]:
for group_id in numpy.unique(k_data):
    getGroupKeyword(group_id, 10)

0 希臘 4.9108234637
0 歐元區 2.33005924094
0 債權 0.669524125324
0 公投 0.658834077342
0 政府 0.655118015554
0 債務 0.63652134643
0 提款 0.621601187768
0 可能 0.582126641745
0 財政部長 0.540912069506
0 櫃員機 0.529374471226
1 最佳 4.04617459463
1 專輯 2.82520550006
1 金曲 1.94067007063
1 蔡依林 1.64389056436
1 國語 1.58337074329
1 有限公司 1.2177551509
1 音樂 1.20907925857
1 張惠妹 1.1492862655
1 開心 1.10664627155
1 演唱 0.963153943651
2 樂園 4.14330231395
2 八仙 4.12942243844
2 新北市 3.93388532716
2 醫院 3.1762003018
2 現場 2.23152989783
2 爆炸 2.22054448284
2 粉塵 1.87660964446
2 彩色 1.82383967746
2 晚間 1.76313521462
2 全力 1.74433563207
3 西瓜 1.66460170799
3 台灣 1.6225689309
3 中國 1.27768694508
3 美國 1.1794085543
3 日本 1.09012430195
3 表示 1.03101524258
3 問題 1.02815489241
3 國民黨 1.01940906769
3 金曲 1.01697909732
3 改革 0.99394349865


## 使用Pandas 讀取SQLIte 資料

In [97]:
import pandas
import sqlite3
with sqlite3.connect('news.sqlite') as db:
    df = pandas.read_sql_query('SELECT * FROM news_entry;', con=db)

In [99]:
# 閱讀前五篇文章
df.head()

,id,category,view_cnt,time,title,summary,link
0,1,生活,0,15:35,【民報書摘】地圖上不存在的國家,本內容由民報提供 (摘錄 全文請連結民報)民報書摘《地圖上不存在的國家》作者：尼克‧米德頓(...,http://www.appledaily.com.tw/realtimenews/arti...
1,2,社會,31324,15:33,【更新】殺1、2人不會判死？ 割喉殺...,（更新：判決理由） 2012年發生的台南湯姆熊男童遭割喉命案，震驚全台，兇嫌曾文欽落網後曾嗆...,http://www.appledaily.com.tw/realtimenews/arti...
2,3,生活,476,15:31,華為徠卡旗艦機 1萬7千元有找,更新：新增影片 華為今在台發表最新旗艦機P9，主打拍照功能，手機搭載兩顆徠卡（Leica）光...,http://www.appledaily.com.tw/realtimenews/arti...
3,4,娛樂,154,15:30,哈孝遠相揪尬籃球 一句話竟然讓JJ硬了！,藝人哈孝遠巧遇懷秋與JJ，並揪他們下次有空一起去打球，懷秋表示興奮！不過JJ有點擔心地說：「...,http://www.appledaily.com.tw/realtimenews/arti...
4,5,國際,0,15:30,【央廣RTI】遊古巴正夯 美國人暴增9...,本內容由中央廣播電臺提供 古巴政府4日表示，2016年1至4月到古巴旅遊的美國...,http://www.appledaily.com.tw/realtimenews/arti...


## 將標題, 分詞結果, 標籤加到各自的List之中

In [226]:
titles = []
corpus = []
tags   = []
for index, news in df.iterrows():
    if news['category'] == '社會' or news['category'] == '娛樂' or news['category'] == '財經':
        titles.append(news['title'])
        tags.append(news['category'])
        article = []
        for word in jieba.cut(news['summary']):
            if re.match('[\u4e00-\u9fa5]+', word):
                article.append(word)
        corpus.append(' '.join(article)) 

In [227]:
from collections import Counter
c = Counter(tags)
c

Counter({'娛樂': 126, '社會': 220, '財經': 134})

## 轉變為詞頻矩陣

In [228]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X    = vectorizer.fit_transform(corpus)
word = vectorizer.get_feature_names()

In [229]:
#word
X.shape

(480, 21239)

## 使用tfidf

In [230]:
'''
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
tfidf       = transformer.fit_transform(X)
#tfidf.toarray()
X = tfidf
'''

'\nfrom sklearn.feature_extraction.text import TfidfTransformer\ntransformer = TfidfTransformer()\ntfidf       = transformer.fit_transform(X)\n#tfidf.toarray()\nX = tfidf\n'

## 分為訓練與測試資料集

In [231]:
from sklearn.cross_validation import train_test_split
train_titles, test_titles, train_data, test_data, train_tags, test_tags= train_test_split(titles, X, tags, test_size=0.30, random_state=42)

In [232]:
len(train_titles)

336

In [233]:
len(test_titles)

144

In [234]:
train_tags[0]

'財經'

## 使用Naive Bayes 建立分類模型

In [235]:
from sklearn.naive_bayes import MultinomialNB
clf=MultinomialNB(alpha=0.01)
clf.fit(train_data,train_tags)

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

## 產生預測結果

In [236]:
pred=clf.predict(test_data)

## 驗證預測結果

In [237]:
from sklearn.metrics import confusion_matrix

print(clf.classes_)
print(confusion_matrix(test_tags,pred))

['娛樂' '社會' '財經']
[[34  0  1]
 [ 0 75  1]
 [ 0  0 33]]


In [238]:
for idx, item in enumerate(zip(test_tags, pred)):
    if item[0] != item[1]:
        print(idx,item[0],item[1], test_titles[idx])

90 社會 財經 【公庫】旗山大溝頂老街 反政府拆遷
126 娛樂 財經 葉樹姍回鍋大愛台　接棒總監


In [239]:
a = [ 1 ,  2 ,  3 ]
b = ['a', 'b', 'c']
for item in zip(a,b):
    print(item)

(1, 'a')
(2, 'b')
(3, 'c')


In [240]:
from sklearn.metrics import accuracy_score
accuracy_score(test_tags,pred)

0.98611111111111116

## 找出電腦分類用的關鍵字

In [241]:
clf.classes_

array(['娛樂', '社會', '財經'], 
      dtype='<U2')

In [242]:
len(clf.coef_)

3

In [245]:
import operator
# 0 => 娛樂 1=> 社會 2=> 財經
word_coef = clf.coef_[0].flatten()
word      = vectorizer.get_feature_names()
word_rank = [item for item in zip(word, word_coef)]
swd       = sorted(word_rank, key=operator.itemgetter(1), reverse=True)
swd[0:30]

[('報導', -5.0449244045611188),
 ('週刊', -5.2962035644495549),
 ('新聞', -5.3788815989255463),
 ('自己', -5.5474726494820628),
 ('粉絲', -5.6106382442361422),
 ('綜合', -5.6106382442361422),
 ('這則', -5.6106382442361422),
 ('節目', -5.6780643616078743),
 ('女兒', -5.7756789848849541),
 ('工作', -5.7756789848849541),
 ('翻攝', -5.8283088620054544),
 ('表示', -5.8557003307427351),
 ('台灣', -5.8838632734369449),
 ('提供', -5.8838632734369449),
 ('台北', -5.9426864625384175),
 ('中國', -5.9734486544217473),
 ('偶像', -5.9734486544217473),
 ('我們', -5.9734486544217473),
 ('不過', -6.0051872752911617),
 ('鹿晗', -6.0051872752911617),
 ('公司', -6.0379663489512438),
 ('娛樂', -6.0718564102702057),
 ('活動', -6.0718564102702057),
 ('網友', -6.0718564102702057),
 ('就是', -6.106935419131502),
 ('鼓鼓', -6.1432898405989942),
 ('香港', -6.2202212635139915),
 ('可以', -6.261026598170357),
 ('因為', -6.261026598170357),
 ('時間', -6.261026598170357)]

## 使用Gensim 建立 word2vec 模型

## 下載Gensim
- https://pypi.python.org/pypi/gensim#downloads
- https://pypi.python.org/packages/74/68/82a3ebb34ac9a7ec4bb004b5a92462e9ce6ce56e8dd7544f913b49bd372a/gensim-0.13.3-cp35-cp35m-win_amd64.whl#md5=9ebf4708ca1cc0306242dd4c0d63d4f1

- conda install win-64-gensim-0.12.4-np111py35_0.tar.bz2

## 下載 smart_open
- https://pypi.python.org/pypi/smart_open/1.3.5
- 解壓縮
- python setup.py install 

In [3]:
import pandas
import sqlite3
with sqlite3.connect('news.sqlite') as db:
    df = pandas.read_sql_query('SELECT * FROM news_entry;', con=db)

In [18]:
import jieba, re

corpus = []
tags = []
title = []
for index, news in df.iterrows():
    #if (rec[2] == '娛樂'.decode('utf-8')):
    ele  = jieba.cut(news['summary'])
    corpus.append([w for w in ' '.join(ele).split()])

In [20]:
from gensim.models import word2vec

In [21]:
num_features = 10
min_word_count = 10
num_workers = 4
context = 10
epoch = 20
sample = 1e-5
model = word2vec.Word2Vec(corpus, workers = num_workers,
                          sample = sample,
                          size = num_features,
                          min_count=min_word_count,
                          window = context,
                          iter = epoch)

In [38]:
[w for w in model.vocab][0:10]

['BBC', '拒絕', '姊', '試驗', '熊本', '多名', '85', '絕食', '差距', '門檻']

In [33]:
for w in model.most_similar('iPhone'):
    print(w[0], w[1])

下載 0.999111533164978
Plus 0.998620331287384
2016 0.9981125593185425
影片 0.9979183077812195
I 0.9978212118148804
— 0.99778813123703
曾雪蒨 0.9977516531944275
警長 0.9976658821105957
Johnson 0.9975661039352417
NASA 0.9974972605705261
